In [ ]:
!pip install rasterio --q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 43.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fclayey.asc to fclayey.asc
Saving fclayskeletal.asc to fclayskeletal.asc
Saving floamy.asc to floamy.asc
Saving fsandy.asc to fsandy.asc
Saving RF25_ind2024_rfp25.nc to RF25_ind2024_rfp25.nc


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving LULCDATA.tif.xml to LULCDATA.tif.xml


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving LULCDATA.tif to LULCDATA.tif


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving GWR_District2020.shp to GWR_District2020.shp
Saving GWR_District2020.shx to GWR_District2020.shx


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Major_Aquifers.shp to Major_Aquifers.shp


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving GWR_District2020.cpg to GWR_District2020.cpg
Saving GWR_District2020.dbf to GWR_District2020.dbf
Saving GWR_District2020.prj to GWR_District2020.prj


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Major_Aquifers.cpg to Major_Aquifers.cpg
Saving Major_Aquifers.dbf to Major_Aquifers.dbf
Saving Major_Aquifers.prj to Major_Aquifers.prj


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Major_Aquifers.shx to Major_Aquifers.shx


In [ ]:
gwr = gpd.read_file("GWR_District2020.shp")
print(gwr.columns)

Index(['District', 'State', 'Annual_Gro', 'Annual_G00', 'Annual_Rep',
       'Natural_Di', 'Projected_', 'Ground_Wat', 'Net_Ground', 'Annual_Dra',
       'Stage_of_d', 'st_area_sh', 'st_length_', 'geometry'],
      dtype='object')


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Load shapefile once (outside the function for efficiency)
gwr = gpd.read_file("GWR_District2020.shp")
drop_cols = ['Annual_G00', 'Natural_Di', 'Net_Ground', 'st_area_sh', 'st_length_']

# Drop columns
gwr = gwr.drop(columns=drop_cols,axis=1)
gwr = gwr.to_crs(epsg=4326)

def get_groundwater_data(lat: float, lon: float):
    """
    Returns the nearest district's groundwater data for given lat/lon.

    Args:
        lat (float): Latitude in decimal degrees
        lon (float): Longitude in decimal degrees

    Returns:
        GeoDataFrame row (nearest district's attributes)
    """
    # Create point GeoDataFrame for input location
    point = gpd.GeoDataFrame(
        [{'geometry': Point(lon, lat)}],
        crs="EPSG:4326"
    )

    # Find nearest district polygon
    nearest = gpd.sjoin_nearest(point, gwr, how="left")

    return nearest


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# -----------------------------
# Load and prepare shapefile
# -----------------------------
aquifers = gpd.read_file("Major_Aquifers.shp")

# Keep only essential columns
essential_cols = ["aquifer", "system", "mbgl", "avg_mbgl",
                  "m2_perday", "m3_per_day", "per_cm", "geometry"]
aquifers = aquifers[essential_cols]

# Reproject to WGS84 (lat/lon) if needed
if aquifers.crs is None or aquifers.crs.to_epsg() != 4326:
    aquifers = aquifers.to_crs(epsg=4326)

# -----------------------------
# Function: Get aquifer info by lat/lon
# -----------------------------
def get_aquifer_info(lat: float, lon: float):
    """
    Find aquifer attributes for a given lat/lon.
    Returns each column value separately.
    """
    point = Point(lon, lat)  # (lon, lat) order

    # Select aquifer polygon containing the point
    result = aquifers[aquifers.contains(point)]

    if result.empty:
        return {
            "message": "No aquifer data found at this location.",
            "aquifer": None,
            "system": None,
            "mbgl": None,
            "avg_mbgl": None,
            "m2_perday": None,
            "m3_per_day": None,
            "per_cm": None
        }

    row = result.iloc[0]

    return {
        "aquifer": row["aquifer"],
        "system": row["system"],
        "mbgl": row["mbgl"],
        "avg_mbgl": row["avg_mbgl"],
        "m2_perday": row["m2_perday"],
        "m3_per_day": row["m3_per_day"],
        "per_cm": row["per_cm"]
    }

In [ ]:
import rasterio
from rasterio.transform import rowcol
from pyproj import Transformer
import numpy as np

# Load raster
lulc_file = "LULCDATA.tif"
with rasterio.open(lulc_file) as src:
    lulc_data = src.read(1)          # read first band
    transform = src.transform        # affine transform
    crs = src.crs                    # get CRS from raster
def get_lulc_class(lat, lon):
    transformer = Transformer.from_crs("EPSG:4326", crs, always_xy=True)
    x, y = transformer.transform(lon, lat)
    row, col = rowcol(transform, x, y)

    row = np.clip(row, 0, lulc_data.shape[0] - 1)
    col = np.clip(col, 0, lulc_data.shape[1] - 1)

    code = int(lulc_data[row, col])
    return {
        "code": code,
        "class": "Urban" if code == 1 else "Rural"
    }

In [ ]:
import os
import numpy as np
import rasterio
from rasterio.crs import CRS
from pyproj import Transformer
from scipy.ndimage import distance_transform_edt
from rasterio.transform import rowcol

# -----------------------------
# STEP 1: Input ASC files
# -----------------------------
soil_files = {
    "Sandy soil": "fsandy.asc",
    "Clay soil": "fclayey.asc",
    "Loamy soil": "floamy.asc",
    "Clayskeletal soil": "fclayskeletal.asc"
}

soil_codes = {
    "Sandy soil": 1,
    "Clay soil": 2,
    "Loamy soil": 3,
    "Clayskeletal soil": 4
}
soil_properties = {
    1: {"porosity": 0.40, "infiltration": 10.0},      # Sandy soil
    2: {"porosity": 0.55, "infiltration": 0.5},       # Clay soil
    3: {"porosity": 0.45, "infiltration": 10.0},      # Loamy soil
    4: {"porosity": 0.50, "infiltration": 2.0}        # Clayskeletal soil
}


output_dir = "geotiff_soils"
os.makedirs(output_dir, exist_ok=True)

# -----------------------------
# STEP 2: Convert ASC -> GeoTIFF with CRS
# -----------------------------
for soil_name, asc_file in soil_files.items():
    with rasterio.open(asc_file) as src:
        profile = src.profile
        profile.update(
            driver="GTiff",
            crs=CRS.from_epsg(32642),  # UTM Zone 42N
            dtype=rasterio.float32,
            nodata=-9999
        )
        out_path = os.path.join(output_dir, soil_name.replace(" ", "_") + ".tif")
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(src.read(1), 1)
        print(f"Converted {asc_file} -> {out_path}")

# -----------------------------
# STEP 3: Combine into one raster
# -----------------------------
ref_file = os.path.join(output_dir, "Sandy_soil.tif")
with rasterio.open(ref_file) as ref:
    profile = ref.profile
    profile.update(dtype=rasterio.uint8, count=1, nodata=0, crs=CRS.from_epsg(32642))
    combined = np.zeros((ref.height, ref.width), dtype=np.uint8)
    transform = ref.transform

for soil_name, code in soil_codes.items():
    tif_file = os.path.join(output_dir, soil_name.replace(" ", "_") + ".tif")
    with rasterio.open(tif_file) as src:
        data = src.read(1)
        mask = (data != src.nodata) & (data > 0)
        combined[mask] = code

combined_path = "combined_soil_map_corrected.tif"
with rasterio.open(combined_path, "w", **profile) as dst:
    dst.write(combined, 1)
print(f"\n✅ Combined raster saved as {combined_path}")

# -----------------------------
# STEP 4: Prepare distance transform for nearest soil lookup
# -----------------------------
with rasterio.open(combined_path) as src:
    combined_data = src.read(1)
    nodata_mask = (combined_data == 0)
    dist, indices = distance_transform_edt(nodata_mask, return_indices=True)

soil_legend = {
    0: "No soil data",
    1: "Sandy soil",
    2: "Clay soil",
    3: "Loamy soil",
    4: "Clayskeletal soil"
}

# -----------------------------
# STEP 5: Function for soil query
# -----------------------------
def get_soil_type(lat: float, lon: float, transformer=None):
    """
    Return soil type at lat/lon, or nearest soil type if NoData.
    Returns a dict (backend-friendly).
    """
    if transformer is None:
        transformer = Transformer.from_crs("EPSG:4326", "EPSG:32642", always_xy=True)
    x, y = transformer.transform(lon, lat)

    # Convert to row/col
    row, col = rowcol(transform, x, y)

    # Clip to raster bounds
    row = np.clip(row, 0, combined_data.shape[0]-1)
    col = np.clip(col, 0, combined_data.shape[1]-1)

    val = combined_data[row, col]
    if val != 0:


        return {
       "latitude": lat,
       "longitude": lon,
       "soil_code": int(val),
       "soil_type": soil_legend[val],
       "porosity": soil_properties[int(val)]["porosity"],
       "infiltration_rate": soil_properties[int(val)]["infiltration"],
       "nearest_used": False
             }

    # If NoData, use nearest soil
    nearest_row = indices[0, row, col]
    nearest_col = indices[1, row, col]
    nearest_val = combined_data[nearest_row, nearest_col]

    return {
      "latitude": lat,
      "longitude": lon,
      "soil_code": int(val),
      "soil_type": soil_legend[val],
      "porosity": soil_properties[int(val)]["porosity"],
      "infiltration_rate": soil_properties[int(val)]["infiltration"],
      "nearest_used": True
          }



Converted fsandy.asc -> geotiff_soils/Sandy_soil.tif
Converted fclayey.asc -> geotiff_soils/Clay_soil.tif
Converted floamy.asc -> geotiff_soils/Loamy_soil.tif
Converted fclayskeletal.asc -> geotiff_soils/Clayskeletal_soil.tif

✅ Combined raster saved as combined_soil_map_corrected.tif


In [ ]:
import numpy as np

def calculate_runoff_coefficient(urban_rural, roof_type, soil_infiltration, soil_porosity):
    """
    Calculate runoff coefficient based on location type, roof type, and soil properties.

    Args:
        urban_rural (str): "Urban" or "Rural"
        roof_type (str): "flat", "tilted", or "sloping"
        soil_infiltration (float): Soil infiltration rate (mm/hr or similar units)
        soil_porosity (float): Soil porosity (0-1)

    Returns:
        float: Runoff coefficient (0-1)
    """

    # Base runoff coefficients for roof types
    roof_coefficients = {
        "flat": 0.85,      # Lower runoff due to potential ponding
        "tilted": 0.90,    # Moderate runoff
        "sloping": 0.95    # High runoff due to steep angle
    }

    # Get base coefficient from roof type
    base_coeff = roof_coefficients.get(roof_type.lower(), 0.90)

    # Urban/Rural adjustment factors
    if urban_rural.lower() == "urban":
        urban_factor = 1.1  # Higher runoff in urban areas
    else:  # Rural
        urban_factor = 0.95  # Slightly lower runoff in rural areas

    # Soil infiltration adjustment (inverse relationship)
    # Higher infiltration = lower runoff coefficient
    if soil_infiltration >= 10.0:  # High infiltration (sandy/loamy)
        infiltration_factor = 0.92
    elif soil_infiltration >= 2.0:  # Medium infiltration
        infiltration_factor = 0.96
    else:  # Low infiltration (clay)
        infiltration_factor = 1.02

    # Soil porosity adjustment (inverse relationship)
    # Higher porosity = lower runoff coefficient
    if soil_porosity >= 0.50:  # High porosity
        porosity_factor = 0.94
    elif soil_porosity >= 0.45:  # Medium porosity
        porosity_factor = 0.97
    else:  # Low porosity
        porosity_factor = 1.01

    # Calculate final coefficient
    runoff_coeff = base_coeff * urban_factor * infiltration_factor * porosity_factor

    # Ensure coefficient stays within realistic bounds (0.7 - 1.0 for roofs)
    runoff_coeff = np.clip(runoff_coeff, 0.70, 1.00)

    return round(runoff_coeff, 3)


def calculate_runoff(lat, lon, roof_type, roof_area, rainfall_intensity, rainfall_duration):
    """
    Calculate total runoff from a roof given location, roof characteristics, and rainfall data.

    Args:
        lat (float): Latitude in decimal degrees
        lon (float): Longitude in decimal degrees
        roof_type (str): "flat", "tilted", or "sloping"
        roof_area (float): Roof area in square meters
        rainfall_intensity (float): Rainfall intensity in mm/hr
        rainfall_duration (float): Rainfall duration in hours

    Returns:
        dict: Comprehensive runoff analysis including all parameters used
    """

    # Get land use classification (urban/rural)
    try:
        lulc_info = get_lulc_class(lat, lon)
        urban_rural = lulc_info["class"]
    except:
        urban_rural = "Rural"  # Default fallback
        print(f"Warning: Could not determine land use for {lat}, {lon}. Using Rural as default.")

    # Get soil properties
    try:
        soil_info = get_soil_type(lat, lon)
        soil_infiltration = soil_info["infiltration_rate"]
        soil_porosity = soil_info["porosity"]
        soil_type = soil_info["soil_type"]
    except:
        # Default soil properties (medium values)
        soil_infiltration = 5.0
        soil_porosity = 0.45
        soil_type = "Unknown soil"
        print(f"Warning: Could not determine soil type for {lat}, {lon}. Using default values.")

    # Calculate runoff coefficient
    runoff_coeff = calculate_runoff_coefficient(urban_rural, roof_type, soil_infiltration, soil_porosity)

    # Calculate total rainfall (mm)
    total_rainfall = rainfall_intensity * rainfall_duration

    # Calculate runoff volume (liters)
    # Formula: Runoff = Coefficient × Rainfall (mm) × Area (m²)
    # Convert mm·m² to liters: 1 mm over 1 m² = 1 liter
    runoff_volume = runoff_coeff * total_rainfall * roof_area

    # Calculate runoff rate (liters per hour during the event)
    runoff_rate = runoff_volume / rainfall_duration if rainfall_duration > 0 else 0

    return {
        "location": {
            "latitude": lat,
            "longitude": lon,
            "land_use": urban_rural
        },
        "roof_characteristics": {
            "roof_type": roof_type,
            "roof_area_m2": roof_area
        },
        "soil_properties": {
            "soil_type": soil_type,
            "infiltration_rate_mm_hr": soil_infiltration,
            "porosity": soil_porosity
        },
        "rainfall_data": {
            "intensity_mm_hr": rainfall_intensity,
            "duration_hours": rainfall_duration,
            "total_rainfall_mm": total_rainfall
        },
        "runoff_results": {
            "runoff_coefficient": runoff_coeff,
            "runoff_volume_liters": round(runoff_volume, 2),
            "runoff_rate_liters_per_hour": round(runoff_rate, 2),
            "collection_efficiency_percent": round(runoff_coeff * 100, 1)
        }
    }


def batch_runoff_analysis(locations_data, rainfall_intensity, rainfall_duration):
    """
    Perform runoff analysis for multiple locations.

    Args:
        locations_data (list): List of dicts with keys: 'lat', 'lon', 'roof_type', 'roof_area'
        rainfall_intensity (float): Rainfall intensity in mm/hr
        rainfall_duration (float): Rainfall duration in hours

    Returns:
        list: List of runoff analysis results for each location
    """
    results = []

    for location in locations_data:
        try:
            result = calculate_runoff(
                lat=location['lat'],
                lon=location['lon'],
                roof_type=location['roof_type'],
                roof_area=location['roof_area'],
                rainfall_intensity=rainfall_intensity,
                rainfall_duration=rainfall_duration
            )
            results.append(result)
        except Exception as e:
            print(f"Error processing location {location['lat']}, {location['lon']}: {str(e)}")
            continue

    return results


# Example usage and testing
if __name__ == "__main__":
    # Example single location analysis
    sample_result = calculate_runoff(
        lat=26.1445,
        lon=91.7362,  # Guwahati coordinates
        roof_type="tilted",
        roof_area=100,  # 100 m² roof
        rainfall_intensity=25,  # 25 mm/hr
        rainfall_duration=2  # 2 hours
    )

    print("Sample Runoff Analysis:")
    print("=" * 50)
    for key, value in sample_result.items():
        if isinstance(value, dict):
            print(f"\n{key.upper().replace('_', ' ')}:")
            for sub_key, sub_value in value.items():
                print(f"  {sub_key.replace('_', ' ').title()}: {sub_value}")
        else:
            print(f"{key.replace('_', ' ').title()}: {value}")

    # Example batch analysis
    sample_locations = [
        {"lat": 26.1445, "lon": 91.7362, "roof_type": "flat", "roof_area": 120},
        {"lat": 26.1500, "lon": 91.7400, "roof_type": "tilted", "roof_area": 80},
        {"lat": 26.1400, "lon": 91.7300, "roof_type": "sloping", "roof_area": 150}
    ]

    batch_results = batch_runoff_analysis(sample_locations, 30, 1.5)
    print(f"\n\nBatch Analysis Results: {len(batch_results)} locations processed")

Sample Runoff Analysis:

LOCATION:
  Latitude: 26.1445
  Longitude: 91.7362
  Land Use: Rural

ROOF CHARACTERISTICS:
  Roof Type: tilted
  Roof Area M2: 100

SOIL PROPERTIES:
  Soil Type: Unknown soil
  Infiltration Rate Mm Hr: 5.0
  Porosity: 0.45

RAINFALL DATA:
  Intensity Mm Hr: 25
  Duration Hours: 2
  Total Rainfall Mm: 50

RUNOFF RESULTS:
  Runoff Coefficient: 0.796
  Runoff Volume Liters: 3980.0
  Runoff Rate Liters Per Hour: 1990.0
  Collection Efficiency Percent: 79.6


Batch Analysis Results: 3 locations processed


In [ ]:
import requests

API_KEY = "579b464db66ec23bdd00000198f041578f1841ed789605d98ab6e940"
RESOURCE_ID = "3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69"

def get_rainfall(lat, lon, date=None, limit=5):
    """
    Fetch rainfall from data.gov.in using the provided resource.
    If lat/lon filtering doesn't work, fallback to general.
    """
    base_url = f"https://api.data.gov.in/resource/{RESOURCE_ID}"

    params = {
        "api-key": API_KEY,
        "format": "json",
        "limit": limit
    }
    if date:
        params["date"] = date

    # Try without lat/lon first (if spatial filtering isn’t supported)
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
    except Exception as e:
        return {"error": f"API request failed: {e}"}

    data = response.json()

    # Check if records are present
    if "records" in data and len(data["records"]) > 0:
        # Optionally, you can pick the one by nearest lat/lon if lat/lon filtering is available in the record fields
        rec = data["records"][0]
        return {
            "rainfall_record": rec
        }
    else:
        return {"error": "No rainfall data found in resource"}

# Example of using it
rain = get_rainfall(26.1445, 91.7362, date="2025-09-11", limit=2)
print(rain)


In [ ]:
#!/usr/bin/env python3
"""
CGWB-Compliant Rainwater Harvesting Recommendation System with API Rainfall Integration
"""

import os
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.transform import rowcol
from pyproj import Transformer
from scipy.ndimage import distance_transform_edt
import requests
from datetime import datetime, timedelta
import calendar

# Global variable to handle JSON null values
null = None

# -----------------------------
# Data Loading and Initialization
# -----------------------------

def initialize_data():
    """Initialize all spatial data with error handling"""
    global gwr, aquifers, lulc_data, transform, crs, combined_data, dist, indices

    try:
        # Load groundwater data
        if os.path.exists("GWR_District2020.shp"):
            gwr = gpd.read_file("GWR_District2020.shp")
            drop_cols = ['Annual_G00', 'Natural_Di', 'Net_Ground', 'st_area_sh', 'st_length_']
            gwr = gwr.drop(columns=[col for col in drop_cols if col in gwr.columns], axis=1)
            gwr = gwr.to_crs(epsg=4326)
        else:
            gwr = None

    except Exception as e:
        print(f"Warning: Could not load groundwater data: {e}")
        gwr = None

    try:
        # Load aquifer data
        if os.path.exists("Major_Aquifers.shp"):
            aquifers = gpd.read_file("Major_Aquifers.shp")
            essential_cols = ["aquifer", "system", "mbgl", "avg_mbgl",
                            "m2_perday", "m3_per_day", "per_cm", "geometry"]
            aquifers = aquifers[[col for col in essential_cols if col in aquifers.columns]]
            if aquifers.crs is None or aquifers.crs.to_epsg() != 4326:
                aquifers = aquifers.to_crs(epsg=4326)
        else:
            aquifers = None
    except Exception as e:
        print(f"Warning: Could not load aquifer data: {e}")
        aquifers = None

    try:
        # Load LULC data
        if os.path.exists("LULCDATA.tif"):
            with rasterio.open("LULCDATA.tif") as src:
                lulc_data = src.read(1)
                transform = src.transform
                crs = src.crs
        else:
            lulc_data = None
            transform = None
            crs = None
    except Exception as e:
        print(f"Warning: Could not load LULC data: {e}")
        lulc_data = None
        transform = None
        crs = None

    try:
        # Load soil data
        if os.path.exists("combined_soil_map_corrected.tif"):
            with rasterio.open("combined_soil_map_corrected.tif") as src:
                combined_data = src.read(1)
                nodata_mask = (combined_data == 0)
                dist, indices = distance_transform_edt(nodata_mask, return_indices=True)
        else:
            combined_data = None
            dist = None
            indices = None
    except Exception as e:
        print(f"Warning: Could not load soil data: {e}")
        combined_data = None
        dist = None
        indices = None

# Initialize data when module is imported
initialize_data()

# -----------------------------
# Rainfall API Integration
# -----------------------------

def get_rainfall_data(lat: float, lon: float, start_date: str = None, end_date: str = None):
    """
    Get rainfall data from API and calculate monthly and annual totals.

    Args:
        lat: Latitude
        lon: Longitude
        start_date: Start date in 'YYYY-MM-DD' format (defaults to 1 year ago)
        end_date: End date in 'YYYY-MM-DD' format (defaults to today)

    Returns:
        dict: Contains monthly and annual rainfall data
    """

    # Set default date range (last 12 months)
    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    if start_date is None:
        start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

    try:
        # API endpoint for historical weather data
        url = "https://api.open-meteo.com/v1/forecast"

        params = {
            'latitude': lat,
            'longitude': lon,
            'daily': 'precipitation_sum',
            'start_date': start_date,
            'end_date': end_date,
            'timezone': 'auto'
        }

        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()

        data = response.json()

        # Extract daily precipitation data
        daily_data = data.get('daily', {})
        dates = daily_data.get('time', [])
        precipitation = daily_data.get('precipitation_sum', [])

        if not dates or not precipitation:
            raise ValueError("No precipitation data received from API")

        # Calculate monthly totals
        monthly_totals = {}
        annual_total = 0.0

        for i, date_str in enumerate(dates):
            if i < len(precipitation) and precipitation[i] is not None:
                date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                month_key = f"{date_obj.year}-{date_obj.month:02d}"
                month_name = f"{calendar.month_name[date_obj.month]} {date_obj.year}"

                if month_key not in monthly_totals:
                    monthly_totals[month_key] = {
                        'month_name': month_name,
                        'total_mm': 0.0,
                        'days_with_rain': 0
                    }

                daily_precip = float(precipitation[i])
                monthly_totals[month_key]['total_mm'] += daily_precip
                annual_total += daily_precip

                if daily_precip > 0.1:  # Count days with meaningful rainfall
                    monthly_totals[month_key]['days_with_rain'] += 1

        # Format monthly data for output
        monthly_rainfall = []
        for month_key in sorted(monthly_totals.keys()):
            month_data = monthly_totals[month_key]
            monthly_rainfall.append({
                'month': month_data['month_name'],
                'total_mm': round(month_data['total_mm'], 1),
                'days_with_rain': month_data['days_with_rain'],
                'average_daily_mm': round(month_data['total_mm'] / calendar.monthrange(
                    int(month_key.split('-')[0]), int(month_key.split('-')[1]))[1], 2)
            })

        # Calculate seasonal distribution
        monsoon_months = [6, 7, 8, 9]  # June to September
        monsoon_total = sum([
            monthly_totals[key]['total_mm'] for key in monthly_totals.keys()
            if int(key.split('-')[1]) in monsoon_months
        ])

        return {
            'location': {
                'latitude': lat,
                'longitude': lon
            },
            'data_period': {
                'start_date': start_date,
                'end_date': end_date,
                'total_days': len(dates)
            },
            'annual_rainfall': {
                'total_mm': round(annual_total, 1),
                'average_daily_mm': round(annual_total / len(dates) if dates else 0, 2)
            },
            'monthly_rainfall': monthly_rainfall,
            'seasonal_analysis': {
                'monsoon_rainfall_mm': round(monsoon_total, 1),
                'monsoon_percentage': round((monsoon_total / annual_total * 100) if annual_total > 0 else 0, 1),
                'non_monsoon_mm': round(annual_total - monsoon_total, 1)
            },
            'rainfall_pattern': {
                'wettest_month': max(monthly_rainfall, key=lambda x: x['total_mm'])['month'] if monthly_rainfall else 'No data',
                'driest_month': min(monthly_rainfall, key=lambda x: x['total_mm'])['month'] if monthly_rainfall else 'No data',
                'rainy_days_total': sum([month['days_with_rain'] for month in monthly_rainfall])
            }
        }

    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return get_fallback_rainfall_data(lat, lon)
    except Exception as e:
        print(f"Error processing rainfall data: {e}")
        return get_fallback_rainfall_data(lat, lon)

def get_fallback_rainfall_data(lat: float, lon: float):
    """
    Provide fallback rainfall data when API fails.
    Based on regional climate patterns in India.
    """

    # Regional rainfall estimates for India
    if 24.0 <= lat <= 28.0 and 89.0 <= lon <= 96.0:
        # Northeast India (Assam region)
        annual_total = 2200
        monsoon_share = 0.75
    elif 8.0 <= lat <= 12.0 and 76.0 <= lon <= 80.0:
        # South India
        annual_total = 1200
        monsoon_share = 0.60
    elif 23.0 <= lat <= 30.0 and 68.0 <= lon <= 78.0:
        # North India
        annual_total = 800
        monsoon_share = 0.70
    else:
        # Default for India
        annual_total = 1100
        monsoon_share = 0.70

    # Generate monthly estimates
    monthly_rainfall = []
    months = [
        'January', 'February', 'March', 'April', 'May', 'June',
        'July', 'August', 'September', 'October', 'November', 'December'
    ]

    # Typical monsoon distribution for India
    monthly_factors = [0.02, 0.02, 0.03, 0.04, 0.06, 0.18, 0.25, 0.22, 0.12, 0.04, 0.01, 0.01]

    for i, month in enumerate(months):
        monthly_mm = annual_total * monthly_factors[i]
        monthly_rainfall.append({
            'month': f"{month} 2024",
            'total_mm': round(monthly_mm, 1),
            'days_with_rain': int(monthly_mm / 10) if monthly_mm > 0 else 0,
            'average_daily_mm': round(monthly_mm / 30, 2)
        })

    monsoon_total = annual_total * monsoon_share

    return {
        'location': {
            'latitude': lat,
            'longitude': lon
        },
        'data_period': {
            'start_date': 'Estimated',
            'end_date': 'Estimated',
            'total_days': 365
        },
        'annual_rainfall': {
            'total_mm': annual_total,
            'average_daily_mm': round(annual_total / 365, 2)
        },
        'monthly_rainfall': monthly_rainfall,
        'seasonal_analysis': {
            'monsoon_rainfall_mm': round(monsoon_total, 1),
            'monsoon_percentage': round(monsoon_share * 100, 1),
            'non_monsoon_mm': round(annual_total - monsoon_total, 1)
        },
        'rainfall_pattern': {
            'wettest_month': 'July 2024',
            'driest_month': 'December 2024',
            'rainy_days_total': 80
        },
        'data_source': 'Regional climate estimates (API unavailable)'
    }

# -----------------------------
# Data Access Functions (unchanged from original)
# -----------------------------

def get_groundwater_data(lat: float, lon: float):
    """Returns the nearest district's groundwater data for given lat/lon."""
    if gwr is None:
        return None

    try:
        point = gpd.GeoDataFrame(
            [{'geometry': Point(lon, lat)}],
            crs="EPSG:4326"
        )
        nearest = gpd.sjoin_nearest(point, gwr, how="left")
        return nearest
    except Exception as e:
        print(f"Error in get_groundwater_data: {e}")
        return None

def get_aquifer_info(lat: float, lon: float):
    """Find aquifer attributes for a given lat/lon."""
    if aquifers is None:
        return {
            "message": "No aquifer data available.",
            "aquifer": None,
            "system": None,
            "mbgl": None,
            "avg_mbgl": None,
            "m2_perday": None,
            "m3_per_day": None,
            "per_cm": None
        }

    try:
        point = Point(lon, lat)
        result = aquifers[aquifers.contains(point)]

        if result.empty:
            return {
                "message": "No aquifer data found at this location.",
                "aquifer": None,
                "system": None,
                "mbgl": None,
                "avg_mbgl": None,
                "m2_perday": None,
                "m3_per_day": None,
                "per_cm": None
            }

        row = result.iloc[0]
        return {
            "aquifer": row.get("aquifer"),
            "system": row.get("system"),
            "mbgl": row.get("mbgl"),
            "avg_mbgl": row.get("avg_mbgl"),
            "m2_perday": row.get("m2_perday"),
            "m3_per_day": row.get("m3_per_day"),
            "per_cm": row.get("per_cm")
        }
    except Exception as e:
        print(f"Error in get_aquifer_info: {e}")
        return {"message": f"Error: {e}"}

def get_lulc_class(lat, lon):
    """Get land use land cover classification"""
    if lulc_data is None or transform is None or crs is None:
        return {"code": 0, "class": "Rural"}  # Default fallback

    try:
        transformer = Transformer.from_crs("EPSG:4326", crs, always_xy=True)
        x, y = transformer.transform(lon, lat)
        row, col = rowcol(transform, x, y)

        row = np.clip(row, 0, lulc_data.shape[0] - 1)
        col = np.clip(col, 0, lulc_data.shape[1] - 1)

        code = int(lulc_data[row, col])
        return {
            "code": code,
            "class": "Urban" if code == 1 else "Rural"
        }
    except Exception as e:
        print(f"Error in get_lulc_class: {e}")
        return {"code": 0, "class": "Rural"}

# Soil properties mapping
soil_properties = {
    1: {"porosity": 0.40, "infiltration": 10.0},      # Sandy soil
    2: {"porosity": 0.55, "infiltration": 0.5},       # Clay soil
    3: {"porosity": 0.45, "infiltration": 10.0},      # Loamy soil
    4: {"porosity": 0.50, "infiltration": 2.0}        # Clayskeletal soil
}

soil_legend = {
    0: "No soil data",
    1: "Sandy soil",
    2: "Clay soil",
    3: "Loamy soil",
    4: "Clayskeletal soil"
}

def get_soil_type(lat: float, lon: float, transformer=None):
    """Return soil type at lat/lon, or nearest soil type if NoData."""
    if combined_data is None:
        return {
            "latitude": lat,
            "longitude": lon,
            "soil_code": 3,  # Default to loamy
            "soil_type": "Loamy soil",
            "porosity": 0.45,
            "infiltration_rate": 10.0,
            "nearest_used": False
        }

    try:
        if transformer is None:
            transformer = Transformer.from_crs("EPSG:4326", "EPSG:32642", always_xy=True)
        x, y = transformer.transform(lon, lat)

        # This assumes transform is available from soil data loading
        row, col = rowcol(transform, x, y)
        row = np.clip(row, 0, combined_data.shape[0]-1)
        col = np.clip(col, 0, combined_data.shape[1]-1)

        val = combined_data[row, col]
        if val != 0 and val in soil_properties:
            return {
                "latitude": lat,
                "longitude": lon,
                "soil_code": int(val),
                "soil_type": soil_legend[val],
                "porosity": soil_properties[int(val)]["porosity"],
                "infiltration_rate": soil_properties[int(val)]["infiltration"],
                "nearest_used": False
            }

        # If NoData, use nearest soil (if indices available)
        if indices is not None:
            nearest_row = indices[0, row, col]
            nearest_col = indices[1, row, col]
            nearest_val = combined_data[nearest_row, nearest_col]

            if nearest_val in soil_properties:
                return {
                    "latitude": lat,
                    "longitude": lon,
                    "soil_code": int(nearest_val),
                    "soil_type": soil_legend[nearest_val],
                    "porosity": soil_properties[int(nearest_val)]["porosity"],
                    "infiltration_rate": soil_properties[int(nearest_val)]["infiltration"],
                    "nearest_used": True
                }

        # Final fallback
        return {
            "latitude": lat,
            "longitude": lon,
            "soil_code": 3,
            "soil_type": "Loamy soil",
            "porosity": 0.45,
            "infiltration_rate": 10.0,
            "nearest_used": False
        }

    except Exception as e:
        print(f"Error in get_soil_type: {e}")
        return {
            "latitude": lat,
            "longitude": lon,
            "soil_code": 3,
            "soil_type": "Loamy soil",
            "porosity": 0.45,
            "infiltration_rate": 10.0,
            "nearest_used": False
        }

# -----------------------------
# Enhanced CGWB Recommendation System with API Rainfall
# -----------------------------

def recommend_recharge_structure_cgwb(lat, lon, roof_area, open_space_area, num_dwellers,
                                      budget_range="medium", usage_intent="household"):
    """
    CGWB-compliant recharge structure recommendation system with API-based rainfall data.

    Args:
        lat: Latitude
        lon: Longitude
        roof_area: Roof catchment area in square meters
        open_space_area: Available open space in square meters
        num_dwellers: Number of household members
        budget_range: "low", "medium", or "high"
        usage_intent: "drinking", "household", "irrigation", "recharge", or "mixed"

    Returns:
        dict: Comprehensive CGWB-compliant recommendations with rainfall analysis
    """

    # Get real rainfall data from API
    print("Fetching rainfall data from API...")
    rainfall_data = get_rainfall_data(lat, lon)
    annual_rainfall = rainfall_data['annual_rainfall']['total_mm']

    # Get environmental data
    try:
        lulc_info = get_lulc_class(lat, lon)
        land_use = lulc_info["class"]

        soil_info = get_soil_type(lat, lon)
        soil_type = soil_info["soil_type"]
        soil_infiltration = soil_info["infiltration_rate"]
        soil_porosity = soil_info["porosity"]

        try:
            gw_info = get_groundwater_data(lat, lon)
            gw_depth = estimate_groundwater_depth(gw_info)
        except:
            gw_depth = 10

        try:
            aquifer_info = get_aquifer_info(lat, lon)
            aquifer_type = aquifer_info.get("system", "Local aquifer")
        except:
            aquifer_type = "Local aquifer"

    except Exception as e:
        print(f"Warning: Using default values due to error: {e}")
        land_use = "Rural"
        soil_type = "Medium soil"
        soil_infiltration = 5.0
        soil_porosity = 0.45
        gw_depth = 10
        aquifer_type = "Local aquifer"

    # Calculate water balance using actual rainfall data
    daily_water_need = calculate_daily_water_need(num_dwellers, usage_intent)
    annual_water_demand = daily_water_need * 365

    # Calculate runoff potential using actual rainfall
    runoff_coeff = 0.85 if land_use == "Urban" else 0.80
    annual_runoff_potential = roof_area * annual_rainfall * runoff_coeff

    # Determine collection period from rainfall pattern
    collection_months = determine_collection_period(rainfall_data['monthly_rainfall'])

    # Get recommendations
    storage_recommendations = get_cgwb_storage_recommendations(
        roof_area, annual_runoff_potential, annual_water_demand, budget_range,
        land_use, usage_intent, gw_depth, open_space_area
    )

    recharge_recommendations = get_cgwb_recharge_recommendations(
        roof_area, open_space_area, soil_infiltration, soil_type, budget_range,
        land_use, gw_depth, annual_runoff_potential
    )

    hybrid_recommendations = get_cgwb_hybrid_recommendations(
        storage_recommendations, recharge_recommendations, budget_range, annual_runoff_potential
    )

    compliance_checklist = generate_cgwb_compliance_checklist(
        roof_area, storage_recommendations, recharge_recommendations
    )

    implementation_plan = generate_implementation_plan(
        storage_recommendations, recharge_recommendations, land_use
    )

    return {
        "location_analysis": {
            "coordinates": {"latitude": lat, "longitude": lon},
            "land_use": land_use,
            "soil_type": soil_type,
            "soil_infiltration_rate": f"{soil_infiltration} mm/hr",
            "groundwater_depth": f"{gw_depth} meters",
            "aquifer_type": aquifer_type
        },

        "rainfall_analysis": {
            "data_source": "Live API data" if 'data_source' not in rainfall_data else rainfall_data.get('data_source'),
            "annual_total": f"{annual_rainfall} mm",
            "monthly_breakdown": rainfall_data['monthly_rainfall'],
            "seasonal_pattern": rainfall_data['seasonal_analysis'],
            "collection_period": collection_months,
            "wettest_month": rainfall_data['rainfall_pattern']['wettest_month'],
            "driest_month": rainfall_data['rainfall_pattern']['driest_month'],
            "total_rainy_days": rainfall_data['rainfall_pattern']['rainy_days_total']
        },

        "water_balance": {
            "household_size": num_dwellers,
            "daily_water_need": f"{daily_water_need} liters",
            "annual_water_demand": f"{annual_water_demand:,.0f} liters",
            "roof_catchment": f"{roof_area} m²",
            "annual_runoff_potential": f"{annual_runoff_potential:,.0f} liters",
            "surplus_deficit_ratio": round((annual_runoff_potential - annual_water_demand) / annual_water_demand, 2),
            "water_self_sufficiency": f"{min(100, (annual_runoff_potential / annual_water_demand) * 100):.1f}%"
        },

        "cgwb_compliant_recommendations": {
            "direct_storage_systems": storage_recommendations,
            "direct_recharge_systems": recharge_recommendations,
            "hybrid_systems": hybrid_recommendations
        },

        "cgwb_compliance": compliance_checklist,
        "implementation_plan": implementation_plan,
        "maintenance_schedule": generate_maintenance_schedule(),
        "regulatory_requirements": get_regulatory_requirements(land_use, roof_area)
    }

# -----------------------------
# Helper Functions
# -----------------------------

def determine_collection_period(monthly_rainfall):
    """Determine the active collection period based on actual monthly rainfall data."""

    significant_months = []
    for month_data in monthly_rainfall:
        if month_data['total_mm'] > 50:  # Months with significant rainfall
            significant_months.append(month_data['month'].split()[0])

    if len(significant_months) >= 6:
        return f"{len(significant_months)} months ({', '.join(significant_months[:3])} to {', '.join(significant_months[-2:])})"
    elif len(significant_months) >= 4:
        return f"{len(significant_months)} months ({significant_months[0]} to {significant_months[-1]})"
    else:
        return f"{len(significant_months)} months (Limited collection period)"

def estimate_groundwater_depth(gw_info):
    """Estimate groundwater depth from available data."""
    return 10  # Default assumption

def calculate_daily_water_need(num_dwellers, usage_intent):
    """Calculate daily water requirement based on usage intent."""
    base_need = num_dwellers * 150

    multipliers = {
        "drinking": 0.2,
        "household": 1.0,
        "irrigation": 1.5,
        "recharge": 1.0,
        "mixed": 1.2
    }

    return base_need * multipliers.get(usage_intent, 1.0)

# -----------------------------
# CGWB System Recommendation Functions (unchanged from original)
# -----------------------------

def get_cgwb_storage_recommendations(roof_area, annual_runoff, annual_demand, budget_range, land_use, usage_intent, gw_depth, open_space_area):
    """Generate CGWB-compliant storage system recommendations with 3-tier pricing."""

    recommendations = {
        "above_ground_tanks": [],
        "underground_cisterns": []
    }

    # Calculate optimal storage capacity
    storage_months = 3 if annual_runoff > annual_demand else 6
    optimal_storage = min(annual_runoff * 0.5, annual_demand * storage_months / 365)

    # Above-Ground Storage Options
    recommendations["above_ground_tanks"] = [
        {
            "tier": "Budget Option",
            "system": "Basic HDPE Tank System",
            "capacity": f"{min(5000, optimal_storage):,.0f} liters",
            "cost_estimate": "₹25,000 - ₹40,000",
            "installation_time": "2-3 days",
            "cgwb_compliance": "Meets basic collection standards"
        },
        {
            "tier": "Standard Option",
            "system": "Ferrocement Tank System",
            "capacity": f"{min(8000, optimal_storage):,.0f} liters",
            "cost_estimate": "₹50,000 - ₹75,000",
            "installation_time": "7-10 days",
            "cgwb_compliance": "Fully compliant with filtration standards"
        },
        {
            "tier": "Premium Option",
            "system": "Smart Stainless Steel System",
            "capacity": f"{min(10000, optimal_storage):,.0f} liters",
            "cost_estimate": "₹90,000 - ₹1,20,000",
            "installation_time": "5-7 days",
            "cgwb_compliance": "Exceeds all quality and monitoring standards"
        }
    ]

    return recommendations

def get_cgwb_recharge_recommendations(roof_area, open_space, soil_infiltration, soil_type, budget_range, land_use, gw_depth, annual_runoff):
    """Generate CGWB-compliant recharge system recommendations."""

    recommendations = {"recharge_pits": []}

    if open_space >= 10:
        num_pits = max(1, min(4, int(roof_area / 100)))

        recommendations["recharge_pits"] = [
            {
                "tier": "Budget Option",
                "system": "Basic Recharge Pits",
                "quantity": f"{num_pits} pits",
                "cost_estimate": f"₹{num_pits * 15000:,} (₹15,000 per pit)",
                "recharge_capacity": f"{num_pits * 5}-{num_pits * 8} L/min",
                "cgwb_compliance": "Meets basic recharge pit standards"
            },
            {
                "tier": "Standard Option",
                "system": "Engineered Recharge Pits",
                "quantity": f"{num_pits} pits",
                "cost_estimate": f"₹{num_pits * 25000:,} (₹25,000 per pit)",
                "recharge_capacity": f"{num_pits * 8}-{num_pits * 12} L/min",
                "cgwb_compliance": "Fully compliant with CGWB technical manual"
            },
            {
                "tier": "Premium Option",
                "system": "Smart Recharge Pit Network",
                "quantity": f"{num_pits + 1} interconnected pits",
                "cost_estimate": f"₹{(num_pits + 1) * 40000:,} (₹40,000 per pit)",
                "recharge_capacity": f"{(num_pits + 1) * 10}-{(num_pits + 1) * 15} L/min",
                "cgwb_compliance": "Exceeds standards with monitoring capabilities"
            }
        ]

    return recommendations

def get_cgwb_hybrid_recommendations(storage_recs, recharge_recs, budget_range, annual_runoff_potential):
    """Combine storage and recharge systems for optimal solutions."""

    return [
        {
            "tier": "Budget Hybrid",
            "system": "Basic Storage + Recharge Combo",
            "cost_estimate": "₹50,000 - ₹70,000",
            "operation": "Store water during early monsoon, recharge excess during peak rainfall",
            "advantages": [
                "Immediate water availability",
                "Groundwater recharge benefit",
                "Seasonal flexibility"
            ]
        },
        {
            "tier": "Standard Hybrid",
            "system": "Integrated Storage-Recharge System",
            "cost_estimate": "₹1,20,000 - ₹1,80,000",
            "operation": "Priority storage fills first, excess automatically diverted to recharge",
            "advantages": [
                "Optimized water use",
                "Automatic overflow management",
                "Year-round benefits"
            ]
        },
        {
            "tier": "Premium Hybrid",
            "system": "Smart Integrated Water Management",
            "cost_estimate": "₹2,50,000 - ₹3,50,000",
            "operation": "AI-optimized water distribution based on weather, usage patterns, and groundwater levels",
            "advantages": [
                "Maximum efficiency",
                "Predictive management",
                "Complete automation",
                "Data-driven optimization"
            ]
        }
    ]

def generate_cgwb_compliance_checklist(roof_area, storage_recs, recharge_recs):
    """Generate CGWB compliance checklist for the recommended systems."""

    return {
        "mandatory_components": {
            "first_flush_diverter": {
                "required": True,
                "compliance_status": "Must be included in all systems"
            },
            "filtration_system": {
                "required": True,
                "compliance_status": "Mandatory for storage systems"
            },
            "overflow_management": {
                "required": True,
                "compliance_status": "Required to prevent waterlogging"
            },
            "inspection_access": {
                "required": True,
                "compliance_status": "Essential for maintenance"
            }
        },
        "design_standards": {
            "recharge_capacity": f"Minimum {roof_area * 0.1 * 0.85:.0f} L/min for peak rainfall (100mm/hr)",
            "storage_sizing": "2-6 months household requirement or 30% of annual runoff",
            "spacing_requirements": "3m from foundation, 1.5m from boundary",
            "depth_limitations": "Maximum 15m for recharge wells without special permission"
        }
    }

def generate_implementation_plan(storage_recs, recharge_recs, land_use):
    """Generate step-by-step implementation plan with timeline."""

    return {
        "pre_implementation_phase": {
            "duration": "2-4 weeks",
            "activities": [
                {
                    "task": "Soil percolation test",
                    "timeline": "Week 1",
                    "requirement": "Mandatory for all recharge systems"
                },
                {
                    "task": "Groundwater quality testing",
                    "timeline": "Week 1-2",
                    "requirement": "Required if existing bore well present"
                },
                {
                    "task": "NOC application to groundwater board",
                    "timeline": "Week 1-3",
                    "requirement": f"{'Mandatory' if land_use == 'Urban' else 'Recommended'}"
                },
                {
                    "task": "Building plan modification/approval",
                    "timeline": "Week 2-4",
                    "requirement": "Required for new constructions"
                },
                {
                    "task": "Contractor selection and quotations",
                    "timeline": "Week 3-4",
                    "requirement": "Use CGWB certified contractors when possible"
                }
            ]
        },
        "best_implementation_timing": {
            "optimal_season": "Pre-monsoon (March-May)",
            "reason": "Allows system commissioning before peak rainfall",
            "avoid_period": "Monsoon season (June-September)"
        }
    }

def generate_maintenance_schedule():
    """Generate CGWB-compliant maintenance schedule."""

    return {
        "monthly_tasks": [
            "Visual inspection of gutters and downpipes",
            "Check first-flush diverter operation",
            "Inspect tank/pit covers for damage"
        ],
        "quarterly_tasks": [
            "Clean filters and screens",
            "Check water quality parameters",
            "Inspect structural integrity"
        ],
        "annual_tasks": [
            "Professional system audit",
            "Deep cleaning of storage tanks",
            "Pump and equipment servicing",
            "Performance evaluation report"
        ],
        "maintenance_costs": {
            "annual": "₹5,000 - ₹15,000 (Complete service)",
            "quarterly": "₹1,500 - ₹3,000 (Routine maintenance)"
        }
    }

def get_regulatory_requirements(land_use, roof_area):
    """Get specific regulatory requirements based on location and system size."""

    base_requirements = {
        "universal_requirements": [
            "Building plan must show RWH provisions",
            "First-flush diverter mandatory",
            "ISI marked materials for permanent installations",
            "Proper overflow arrangements required"
        ]
    }

    if land_use == "Urban":
        base_requirements["urban_specific"] = [
            "NOC from local groundwater authority",
            "Municipal building plan approval",
            "Fire safety compliance for storage systems"
        ]

    if roof_area > 300:
        base_requirements["large_system_requirements"] = [
            "Professional design certification",
            "Environmental impact assessment",
            "Regular monitoring and reporting"
        ]

    return base_requirements

# -----------------------------
# Runoff Calculation Functions (unchanged from original)
# -----------------------------

def calculate_runoff_coefficient(urban_rural, roof_type, soil_infiltration, soil_porosity):
    """Calculate runoff coefficient based on location and soil properties."""

    roof_coefficients = {
        "flat": 0.85,
        "tilted": 0.90,
        "sloping": 0.95
    }

    base_coeff = roof_coefficients.get(roof_type.lower(), 0.90)

    # Urban/Rural adjustment
    urban_factor = 1.1 if urban_rural.lower() == "urban" else 0.95

    # Infiltration adjustment
    if soil_infiltration >= 10.0:
        infiltration_factor = 0.92
    elif soil_infiltration >= 2.0:
        infiltration_factor = 0.96
    else:
        infiltration_factor = 1.02

    # Porosity adjustment
    if soil_porosity >= 0.50:
        porosity_factor = 0.94
    elif soil_porosity >= 0.45:
        porosity_factor = 0.97
    else:
        porosity_factor = 1.01

    runoff_coeff = base_coeff * urban_factor * infiltration_factor * porosity_factor
    runoff_coeff = np.clip(runoff_coeff, 0.70, 1.00)

    return round(runoff_coeff, 3)

def calculate_runoff(lat, lon, roof_type, roof_area, rainfall_intensity, rainfall_duration):
    """Calculate total runoff from a roof given location and rainfall data."""

    try:
        lulc_info = get_lulc_class(lat, lon)
        urban_rural = lulc_info["class"]
    except:
        urban_rural = "Rural"

    try:
        soil_info = get_soil_type(lat, lon)
        soil_infiltration = soil_info["infiltration_rate"]
        soil_porosity = soil_info["porosity"]
        soil_type = soil_info["soil_type"]
    except:
        soil_infiltration = 5.0
        soil_porosity = 0.45
        soil_type = "Unknown soil"

    runoff_coeff = calculate_runoff_coefficient(urban_rural, roof_type, soil_infiltration, soil_porosity)
    total_rainfall = rainfall_intensity * rainfall_duration
    runoff_volume = runoff_coeff * total_rainfall * roof_area
    runoff_rate = runoff_volume / rainfall_duration if rainfall_duration > 0 else 0

    return {
        "location": {
            "latitude": lat,
            "longitude": lon,
            "land_use": urban_rural
        },
        "roof_characteristics": {
            "roof_type": roof_type,
            "roof_area_m2": roof_area
        },
        "soil_properties": {
            "soil_type": soil_type,
            "infiltration_rate_mm_hr": soil_infiltration,
            "porosity": soil_porosity
        },
        "rainfall_data": {
            "intensity_mm_hr": rainfall_intensity,
            "duration_hours": rainfall_duration,
            "total_rainfall_mm": total_rainfall
        },
        "runoff_results": {
            "runoff_coefficient": runoff_coeff,
            "runoff_volume_liters": round(runoff_volume, 2),
            "runoff_rate_liters_per_hour": round(runoff_rate, 2),
            "collection_efficiency_percent": round(runoff_coeff * 100, 1)
        }
    }

# -----------------------------
# Example Usage and Testing Function
# -----------------------------

def test_system():
    """Test function to demonstrate the enhanced system with API rainfall integration."""

    # Example coordinates (Guwahati, Assam)
    test_lat = 26.1445
    test_lon = 91.7362

    print("Testing Enhanced CGWB Rainwater Harvesting System")
    print("=" * 60)

    # Test rainfall data fetch
    print("\n1. Testing Rainfall Data Fetch:")
    rainfall_data = get_rainfall_data(test_lat, test_lon)
    print(f"   Annual Rainfall: {rainfall_data['annual_rainfall']['total_mm']} mm")
    print(f"   Monsoon Share: {rainfall_data['seasonal_analysis']['monsoon_percentage']}%")

    # Test comprehensive recommendation
    print("\n2. Testing Comprehensive Recommendation:")
    recommendations = recommend_recharge_structure_cgwb(
        lat=test_lat,
        lon=test_lon,
        roof_area=150,  # 150 sq meters
        open_space_area=50,  # 50 sq meters
        num_dwellers=4,
        budget_range="medium",
        usage_intent="household"
    )

    print(f"   Location: {recommendations['location_analysis']['land_use']} area")
    print(f"   Soil Type: {recommendations['location_analysis']['soil_type']}")
    print(f"   Water Self-Sufficiency: {recommendations['water_balance']['water_self_sufficiency']}")
    print(f"   Collection Period: {recommendations['rainfall_analysis']['collection_period']}")

    print("\n3. Available System Options:")
    storage_options = recommendations['cgwb_compliant_recommendations']['direct_storage_systems']['above_ground_tanks']
    for option in storage_options:
        print(f"   {option['tier']}: {option['system']} - {option['cost_estimate']}")

    return recommendations

if __name__ == "__main__":
    # Test with typical Guwahati household
    result = recommend_recharge_structure_cgwb(
        lat=26.1445,
        lon=91.7362,
        roof_area=150,
        open_space_area=100,
        num_dwellers=4,
        budget_range="medium",
        usage_intent="household",
        #annual_rainfall=2200
    )

    print("CGWB-Compliant Rainwater Harvesting Recommendations")
    print("=" * 60)

    # Location Analysis
    print("\n LOCATION ANALYSIS:")
    location = result["location_analysis"]
    print(f"Coordinates: {location['coordinates']['latitude']}, {location['coordinates']['longitude']}")
    print(f"Land Use: {location['land_use']}")
    print(f"Soil Type: {location['soil_type']}")
    print(f"Soil Infiltration: {location['soil_infiltration_rate']}")
    print(f"Groundwater Depth: {location['groundwater_depth']}")
   # print(f"Annual Rainfall: {location['annual_rainfall']}")

    # Water Balance
    print("\n WATER BALANCE:")
    water = result["water_balance"]
    print(f"Household Size: {water['household_size']} people")
    print(f"Daily Water Need: {water['daily_water_need']}")
    print(f"Annual Water Demand: {water['annual_water_demand']}")
    print(f"Roof Catchment Area: {water['roof_catchment']}")
    print(f"Annual Runoff Potential: {water['annual_runoff_potential']}")
    print(f"Surplus/Deficit Ratio: {water['surplus_deficit_ratio']}")
   # print(f"Collection Period: {water['collection_months']}")

    # Storage Systems
    print("\n STORAGE SYSTEM RECOMMENDATIONS:")
    storage = result["cgwb_compliant_recommendations"]["direct_storage_systems"]

    if storage.get("above_ground_tanks"):
        print("\n  Above-Ground Tanks:")
        for tank in storage["above_ground_tanks"]:
            print(f"    {tank['tier']}: {tank['system']}")
            print(f"    Capacity: {tank['capacity']}")
            print(f"    Cost: {tank['cost_estimate']}")
            print(f"    Installation: {tank['installation_time']}")
            print(f"    Compliance: {tank['cgwb_compliance']}")
            print()

    if storage.get("underground_cisterns"):
        print("  Underground Cisterns:")
        for cistern in storage["underground_cisterns"]:
            print(f"    {cistern['tier']}: {cistern['system']}")
            print(f"    Capacity: {cistern['capacity']}")
            print(f"    Cost: {cistern['cost_estimate']}")
            print(f"    Installation: {cistern['installation_time']}")
            print()

    # Recharge Systems
    print("RECHARGE SYSTEM RECOMMENDATIONS:")
    recharge = result["cgwb_compliant_recommendations"]["direct_recharge_systems"]

    if recharge.get("recharge_pits"):
        print("\n  Recharge Pits:")
        for pit in recharge["recharge_pits"]:
            print(f"    {pit['tier']}: {pit['system']}")
            print(f"    Quantity: {pit['quantity']}")
            print(f"    Cost: {pit['cost_estimate']}")
            print(f"    Recharge Capacity: {pit['recharge_capacity']}")
            print(f"    Compliance: {pit['cgwb_compliance']}")
            print()

    if recharge.get("recharge_wells"):
        print("  Recharge Wells:")
        for well in recharge["recharge_wells"]:
            print(f"    {well['tier']}: {well['system']}")
            print(f"    Quantity: {well['quantity']}")
            print(f"    Cost: {well['cost_estimate']}")
            print(f"    Recharge Capacity: {well['recharge_capacity']}")
            print()

    # Hybrid Systems
    print(" HYBRID SYSTEM RECOMMENDATIONS:")
    hybrid = result["cgwb_compliant_recommendations"]["hybrid_systems"]
    for system in hybrid:
        print(f"\n  {system['tier']}: {system['system']}")
        print(f"    Cost: {system['cost_estimate']}")
        print(f"    Operation: {system['operation']}")
        print(f"    Advantages: {', '.join(system['advantages'])}")
        print()

    # CGWB Compliance
    print(" CGWB COMPLIANCE REQUIREMENTS:")
    compliance = result["cgwb_compliance"]

    print("\n  Mandatory Components:")
    for component, details in compliance["mandatory_components"].items():
        print(f"    {component.replace('_', ' ').title()}: {details['compliance_status']}")

    print("\n  Design Standards:")
    for standard, requirement in compliance["design_standards"].items():
        print(f"    {standard.replace('_', ' ').title()}: {requirement}")

    # Implementation Plan
    print("\n IMPLEMENTATION PLAN:")
    implementation = result["implementation_plan"]

    print(f"\n  Pre-Implementation ({implementation['pre_implementation_phase']['duration']}):")
    for activity in implementation["pre_implementation_phase"]["activities"]:
        print(f"    {activity['timeline']}: {activity['task']}")

    print(f"\n  Best Implementation Timing:")
    timing = implementation["best_implementation_timing"]
    print(f"    Optimal Season: {timing['optimal_season']}")
    print(f"    Reason: {timing['reason']}")
    print(f"    Avoid: {timing['avoid_period']}")

    # Maintenance Schedule
    print("\n MAINTENANCE REQUIREMENTS:")
    maintenance = result["maintenance_schedule"]

    print("  Monthly Tasks:")
    for task in maintenance["monthly_tasks"][:3]:  # Show first 3 tasks
        print(f"    • {task}")

    print(f"  Annual Maintenance Cost: {maintenance['maintenance_costs']['annual']}")

    # Regulatory Requirements
    print("\n REGULATORY REQUIREMENTS:")
    regulatory = result["regulatory_requirements"]

    print("  Universal Requirements:")
    for req in regulatory["universal_requirements"][:3]:  # Show first 3
        print(f"    • {req}")

    if "urban_specific" in regulatory:
        print("  Urban-Specific Requirements:")
        for req in regulatory["urban_specific"][:2]:  # Show first 2
            print(f"    • {req}")

    print("\n" + "=" * 60)
    print(" RECOMMENDATION SUMMARY:")
    print("Based on your location and requirements, we recommend starting with")
    print("a Standard Hybrid system combining 8,000L underground storage with")
    print("engineered recharge pits for optimal water management and CGWB compliance.")

    print(f"\n ESTIMATED TOTAL COST: ₹1,20,000 - ₹1,80,000")
    print(f" IMPLEMENTATION TIME: 3-4 weeks")
    print(f" WATER BENEFIT: {water['annual_runoff_potential']} potential annual collection")